In [1]:
#### Read Inputs #### 

import xpress as xp
import pandas as pd
import numpy as np
import random
import math

# pd.set_option('display.max_columns', None)
# pd.options.mode.chained_assignment = None  # default='warn'. Uncomment this to remove warning messages.

customer = pd.read_excel('231013_Customer_Base.xlsx')
fraud = pd.read_excel('231013_Fraud_Cases.xlsx')
transact = pd.read_excel('231013_Transactions_Input.xlsx')
customer2 = pd.read_excel('231114_Customer_Base_2nd_batch.xlsx')
customer2 = customer2.rename(columns={"bank_to" : "home_bank"}) # Rename column name 'bank_to' to 'home_bank' to be consistent
fraud2 = pd.read_excel('231114_Fraud_Cases_2nd_batch.xlsx')
transact2 = pd.read_excel('231114_Transactions_input_2nd_batch.xlsx')

In [2]:
#### Data Cleaning ####
# Merging Transaction dataset with Customer dataset by customer_id
full = transact.merge(customer, how = 'left', on='customer_id')
# Remove all cash withdrawals, and all "paid_in" data from the dataset
cleaned = full[(full.category != "Cash Withdrawal") & (full.In_or_Out != "paid_in")]
cleaned["is_scam"] = cleaned["transaction_id"].isin(fraud["transaction_id"]) # Making boolean of True or False if its in the fraud database
cleaned["is_scam"] = cleaned["is_scam"].astype(int) # Converting boolean into 0 and 1

one_hot = pd.get_dummies(cleaned['category']) # Turn the categories into dummies
one_hot = one_hot.astype(int) # Change dummies into integers
full = cleaned.join(one_hot) # Add Dummies to full dataset
full = full.reset_index(drop = True) # Reset index of full dataset

full["day"] = (full["date"] - full["date"][0]).dt.days # "day" indicates which day

diction = {1: 0.25, # To convert priority into time spent investigating
           2: 0.5,
           3: 1,
           4: 2
}

full["time_spent"] = full["priority"] # Temporary
full = full.replace({"time_spent":diction}) # Replace priority with time spent using dictionary

# Drop streaming services from the dataset

####################################################################################################################


# Merging Transaction dataset with Customer dataset by customer_id
full2 = transact2.merge(customer2, how = 'left', on='customer_id')
# Remove all cash withdrawals, and all "paid_in" data from the dataset
cleaned2 = full2[(full2.category != "Cash Withdrawal") & (full2.In_or_Out != "paid_in")]
cleaned2["is_scam"] = cleaned2["transaction_id"].isin(fraud2["transaction_id"])
cleaned2["is_scam"] = cleaned2["is_scam"].astype(int)
one_hot2 = pd.get_dummies(cleaned2['category'])
one_hot2 = one_hot2.astype(int)
full2 = cleaned2.join(one_hot2)
full2 = full2.reset_index(drop = True)

full2["day"] = (full2["date"] - full2["date"][0]).dt.days

n_categories2 = np.shape(one_hot2)[1]
Categories2 = range(n_categories2)

full2["time_spent"] = full2["priority"]
full2 = full2.replace({"time_spent":diction})

n_daily_cases2 = np.shape(full2)[0]

#### NOTE! No Streaming Services category in dataset 2. Hence create a column of 0s in full2
full2["Streaming Services"] = 0
cols = full2.columns.tolist()
cols = cols[0:33] + cols[-1:] + cols[33:-1]
cols
full2 = full2[cols]

C:\Users\yeote\AppData\Local\Temp\ipykernel_8128\1908735088.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned["is_scam"] = cleaned["transaction_id"].isin(fraud["transaction_id"]) # Making boolean of True or False if its in the fraud database
C:\Users\yeote\AppData\Local\Temp\ipykernel_8128\1908735088.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned["is_scam"] = cleaned["is_scam"].astype(int) # Converting boolean into 0 and 1
C:\Users\yeote\AppData\Local\Temp\ipykernel_8128\1908735088.py

In [21]:
full2.loc[full2["is_scam"] == 1].groupby(["bank_from"])["Amount"].sum().to_numpy()

array([ 60856.95,  73087.3 , 102035.2 ,  58804.5 ,  55544.3 ])

In [22]:
test10 = np.array([1,2,3,4])
test11 = np.array([5,6,7,8])
test10 + test11

array([ 6,  8, 10, 12])

In [32]:
test12 = pd.DataFrame([1,2,[4,3]])
test12
test12.to_csv("testing12.csv", index = False)

In [36]:
pd.DataFrame(full2.loc[full2["is_scam"] == 1]["transaction_id"].to_numpy())

,0
0,315355
1,315587
2,315784
3,315809
4,315977
...,...
1001,475334
1002,475614
1003,475624
1004,475893


In [116]:
2 in np.array([full2.loc[full2["day"] == 1 ,"priority"]])

True

In [117]:
# Now we want to find the time taken to investigate any given case, based on their priority and based on whether it is shared.
temp = np.zeros((np.shape(full)[0], 6))
temp = pd.DataFrame(temp, columns = ['bank_A', 'bank_B', 'bank_C', 'bank_D', 'bank_E', 'Intrnl']) 
bank_to_df = temp.reset_index()
bank_to_df.update(full.pivot(columns = 'bank_to', values = 'time_spent')) # Pivot to get the time spent for each bank_to
bank_to_df = bank_to_df.set_index('index').rename_axis(None)
bank_to_df

bank_from_df = temp.reset_index()
bank_from_df.update(full.pivot(columns = 'bank_from', values = 'time_spent')) # Pivot to get the time spent for each bank_from
bank_from_df = bank_from_df.set_index('index').rename_axis(None)

ultimate = bank_to_df/2 + bank_from_df/2 # Halve and add them: this forces the 50:50 split for sharing cases

ultimate.loc[ultimate["Intrnl"] > 0, :] *= 2 # Multiply the time taken by 2 for any bank which investigates alone when bank_to is International

full = full.join(ultimate) # Combine with full dataset

####################################################################################################################

temp2 = np.zeros((np.shape(full2)[0], 6))
temp2 = pd.DataFrame(temp2, columns = ['bank_A', 'bank_B', 'bank_C', 'bank_D', 'bank_E', 'Intrnl']) 
bank_to_df = temp2.reset_index()
bank_to_df.update(full2.pivot(columns = 'bank_to', values = 'time_spent')) # Pivot to get the time spent for each bank_to
bank_to_df = bank_to_df.set_index('index').rename_axis(None)
bank_to_df
bank_from_df = temp2.reset_index()
bank_from_df.update(full2.pivot(columns = 'bank_from', values = 'time_spent')) # Pivot to get the time spent for each bank_from
bank_from_df = bank_from_df.set_index('index').rename_axis(None)
ultimate2 = bank_to_df/2 + bank_from_df/2 # Halve and add them: this forces the 50:50 split for sharing cases
ultimate2.loc[ultimate2["Intrnl"] > 0, :] *= 2 # Multiply the time taken by 2 for any bank which investigates alone when bank_to is International
full2 = full2.join(ultimate2) # Combine with full dataset

####################################################################################################################

# Data Initialisation
teamsize = [8, 12, 10, 10, 10]
bank_names = ['bank_A', 'bank_B', 'bank_C', 'bank_D', 'bank_E']

time = [0.25, 0.5, 1, 2]
ext_cost = [40, 60, 100, 150]

n_priority = len(time)
Priorities = range(n_priority)

# Index Sets
n_banks = 5
Banks = range(n_banks)

Categories = range(np.shape(one_hot)[1])

## Combined mega dataframe ##
training = pd.concat([full, full2], ignore_index = True)
training["day"] = (training["date"] - training["date"][0]).dt.days


In [140]:
print(np.shape(full)[1])
test = pd.DataFrame(0, index = np.arange(3), columns=full.columns)
test.at[np.shape(test)[0]-1,"category"] = 1
pd.concat([test,pd.DataFrame(0, index = np.arange(3), columns=full.columns)], ignore_index = True)

45


,transaction_id,description,Amount,category,date,month,customer_id,type,In_or_Out,bank_to,...,Transportation,Utilities,day,time_spent,bank_A,bank_B,bank_C,bank_D,bank_E,Intrnl
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
# Begin loop
n_training_days = len(np.unique(full["day"]))
# n_testing_days = len(np.unique(full2["day"]))
n_testing_days = 2
q = 1
#random.seed(4)
# for q in range(n_testing_days):
test_day = n_training_days + q

full = training.loc[training["day"] < test_day].reset_index(drop = True)

# full2 is testing dataset
full2 = training.loc[training["day"] == test_day].reset_index(drop = True)

# Index sets
n_days = len(pd.unique(full["date"]))
Days = range(n_days)
n_daily_cases = np.shape(full)[0]
Cases = range(n_daily_cases)


prob_full = xp.problem('prob_full')
prob_full.setControl('outputlog', 0) # suppress output

weight_f = np.array([xp.var(vartype = xp.continuous, name = 'weight_{0}'.format(i+1), lb = -xp.infinity) 
                    for i in Categories], dtype = xp.npvar) # now with negative weights possible. Later defined to be <= 0.9

transact_w_f = xp.var(vartype = xp.continuous, name='transact_w_f') # 0 to 1
cust_w_f = xp.var(vartype = xp.continuous, name='cust_w_f') # 0 to 1
amount_w_f = xp.var(vartype = xp.continuous, name='amount_w_f')

combined = np.array([xp.var(vartype = xp.continuous, name = 'combined_{0}'.format(i+1))
            for i in Cases], dtype = xp.npvar) # 0 to 1


internal = np.array([xp.var(vartype = xp.continuous, name = 'internal_{0}_{1}_{2}'.format(i+1, j+1, k+1))
            for i in Days for j in Banks for k in Priorities], dtype = xp.npvar).reshape(n_days, n_banks, n_priority) # Expected no. of days of internal investigatins
external = np.array([xp.var(vartype = xp.continuous, name = 'external_{0}_{1}_{2}'.format(i+1, j+1, k+1))
            for i in Days for j in Banks for k in Priorities], dtype = xp.npvar).reshape(n_days, n_banks, n_priority) # Expected no. of external investigators of that priority.


prob_full.addVariable(weight_f, cust_w_f, transact_w_f, amount_w_f, combined, internal, external) 

# Constraints

# Combined weights
combined_cons = [combined[i] == xp.Sum(weight_f[c] * full.iloc[i, 17+c] for c in Categories) + transact_w_f * full.iloc[i,11] + cust_w_f * full.iloc[i,15] + amount_w_f * full.iloc[i,2] for i in Cases]
# Does this really need to be in a constraint??

probab_invst_cons = [combined[i] <= 1 for i in Cases] # 0 <= Combined weights <= 1
prob_full.addConstraint(combined_cons, probab_invst_cons) 
for i in Cases:
    for b in Banks:
        full.iloc[i, 39+b] = full.iloc[i, 39+b] * combined[i] # Multiplying decision variable with time spent.
# Any way to speed up the above?


In [65]:
# Find sum by day by bank by priority
temptemptemp = full.groupby(["priority", "day"])
A_df = pd.DataFrame({'bank_A': temptemptemp["bank_A"].sum()}).reset_index()
B_df = pd.DataFrame({'bank_B': temptemptemp["bank_B"].sum()}).reset_index()
C_df = pd.DataFrame({'bank_C': temptemptemp["bank_C"].sum()}).reset_index()
D_df = pd.DataFrame({'bank_D': temptemptemp["bank_D"].sum()}).reset_index()
E_df = pd.DataFrame({'bank_E': temptemptemp["bank_E"].sum()}).reset_index()

A_df["bank_B"] = B_df["bank_B"]
A_df["bank_C"] = C_df["bank_C"]
A_df["bank_D"] = D_df["bank_D"]
A_df["bank_E"] = E_df["bank_E"]
exp_work_df = A_df

for d in Days:
    for b in Banks:
        for p in Priorities:
            prob_full.addConstraint(exp_work_df.iloc[(p-1+1)*(n_days) + d, b+2] <= internal[d,b,p] + (external[d,b,p] * time[p])) # Time constraint for expected investigation

for d in Days:
    for b in Banks:
        prob_full.addConstraint(xp.Sum(internal[d,b,p] for p in Priorities) == teamsize[b]) # All internal investigation days for each bank mus be less than their team size.

for c in Categories:
    prob_full.addConstraint(weight_f[c] <= 0.9) # Upper bound on weights as 0.9

# Objective and solve
prob_full.setObjective(xp.Sum(external[d,b,p] * ext_cost[p] for d in Days for b in Banks for p in Priorities if d < (test_day - 10)) + 
                        xp.Sum((1-combined[i]) * full.iloc[i,16] * full.iloc[i,2] for i in Cases if full["day"][i] < (test_day - 10)) +
                        30 * xp.Sum(external[d,b,p] * ext_cost[p] for d in Days for b in Banks for p in Priorities if d >= (test_day - 10)) + 
                        30 * xp.Sum((1-combined[i]) * full.iloc[i,16] * full.iloc[i,2] for i in Cases if full["day"][i] >= (test_day - 10)), sense = xp.minimize)
prob_full.solve()
print(f'The objective function value is {prob_full.getObjVal()}')

transact_sol = prob_full.getSolution(transact_w_f)
cust_sol = prob_full.getSolution(cust_w_f)
weight_sol = prob_full.getSolution(weight_f)
amount_sol = prob_full.getSolution(amount_w_f)
print("The weights on the categories are: ", weight_sol)
print("The weights on the transaction probabilities are: ", transact_sol)
print("The weights on the customer probabilities are: ", cust_sol)
print("The weights on the transaction value are: ", amount_sol)

print(prob_full.getSolution(combined))
print(prob_full.getSolution(internal))
print(prob_full.getSolution(external))


vec = [amount_sol, transact_sol, cust_sol]
vec_mult_frame = full2.iloc[:,[2,11,15]] * vec
weight_sol_proc_mult_frame = full2.iloc[:,17:37] * weight_sol
joined_mult_frame = vec_mult_frame.join(weight_sol_proc_mult_frame)

combined_day2 = joined_mult_frame.sum(axis = 1)
combined_day2.loc[combined_day2 > 1] = 1
combined_day2.loc[combined_day2 < 0] = 0
# sample from combined_day2

sample = np.random.binomial(1,combined_day2)
to_investigate = [i for i in range(len(sample)) if sample[i]==1]
full2['to_investigate'] = sample
# full2.loc[full2["to_investigate"] == 1] # Uncomment to see all cases investigated

#Scams detected:
print(full2[(full2["to_investigate"] == 1) & (full2["is_scam"] == 1)]) # Uncomment to find scams detected

# Find sum by bank by priority
temptemptemp2 = full2.loc[full2["to_investigate"]==1].groupby(["priority", "day"])
A_df = pd.DataFrame({'bank_A': temptemptemp2["bank_A"].sum()}).reset_index()
B_df = pd.DataFrame({'bank_B': temptemptemp2["bank_B"].sum()}).reset_index()
C_df = pd.DataFrame({'bank_C': temptemptemp2["bank_C"].sum()}).reset_index()
D_df = pd.DataFrame({'bank_D': temptemptemp2["bank_D"].sum()}).reset_index()
E_df = pd.DataFrame({'bank_E': temptemptemp2["bank_E"].sum()}).reset_index()

A_df["bank_B"] = B_df["bank_B"]
A_df["bank_C"] = C_df["bank_C"]
A_df["bank_D"] = D_df["bank_D"]
A_df["bank_E"] = E_df["bank_E"]
exp_work_df2 = A_df
print(exp_work_df2)
subtract_exp_work_df2 = exp_work_df2

# Subtract from low priority first
# WARNING this code assumes that there's only one day being investigated
for b in Banks:
    remaining = teamsize[b]
    for p in range(np.shape(subtract_exp_work_df2)[0]): # Loop over priorities; this is necessary because sometimes the model doesnt investigate prio 4
        if (remaining > 0):
            if (subtract_exp_work_df2.iloc[p,2+b] - remaining < 0):
                remaining = remaining - subtract_exp_work_df2.iloc[p,2+b]
                subtract_exp_work_df2.iloc[p,2+b] = 0
            else: 
                subtract_exp_work_df2.iloc[p,2+b] = subtract_exp_work_df2.iloc[p,2+b] - remaining
                remaining = 0
print(subtract_exp_work_df2)
spending = 0
for p in range(np.shape(subtract_exp_work_df2)[0]): # Loop over priorities; this is necessary because sometimes the model doesnt investigate prio 4
    spending += sum(np.ceil(subtract_exp_work_df2.iloc[p, 2:7] / time[p])) * ext_cost[p]
spending

# Amount gained from detecting scams
print("Gain from the detected scams is: ", sum(full2[(full2["to_investigate"] == 1) & (full2["is_scam"] == 1)]["Amount"]))

# Actual loss from undetected scams
print("Loss from scams undetected is: ", sum(full2[(full2["to_investigate"] == 0) & (full2["is_scam"] == 1)]["Amount"]))

# Total loss after accounting for external investigators
print("Total loss + spending: ",  sum(full2[(full2["to_investigate"] == 0) & (full2["is_scam"] == 1)]["Amount"]) + spending)

altspending = 0
for p in range(np.shape(subtract_exp_work_df2)[0]): # Loop over priorities; this is necessary because sometimes the model doesnt investigate prio 4
    altspending += np.ceil(sum(subtract_exp_work_df2.iloc[p, 2:7] / time[p])) * ext_cost[p]

# Better loss if we better allocate external investigators
print("Better total loss + spending by better allocating external investigators: ",  sum(full2[(full2["to_investigate"] == 0) & (full2["is_scam"] == 1)]["Amount"]) + altspending)



The objective function value is 273340.9579441475
The weights on the categories are:  [-0.00452717 -0.00538728 -0.00616325 -0.00443368 -0.00985528  0.9
 -0.00538728  0.05700593  0.70524004 -0.00708506 -0.04914074  0.9
 -0.01548423 -0.00475117 -0.00584082  0.9        -0.00475117 -0.00950167
 -0.00577807 -0.00519155]
The weights on the transaction probabilities are:  0.0
The weights on the customer probabilities are:  0.021326972015241235
The weights on the transaction value are:  3.739611858729453e-05
[0.0131779  0.00750244 0.01441788 ... 0.00106635 0.01185864 0.0029013 ]
[[[2.33954605 1.72850532 1.74444272 2.18750591]
  [2.58559789 0.34544063 1.78021037 7.28875112]
  [3.67762749 0.32180432 5.97591855 0.02464963]
  [6.29109997 0.81416955 2.84598656 0.04874392]
  [2.44339143 0.81793508 2.48267501 4.25599848]]

 [[2.05653714 0.8332784  0.63610335 4.47408111]
  [2.55175638 0.63440008 0.47812131 8.33572223]
  [1.60807905 0.76815205 1.31769555 6.30607334]
  [2.13481899 0.59122645 7.25174382 

In [66]:
candidate_cases = full2[full2["to_investigate"] == 1]
n_daily_cases3 = np.shape(candidate_cases)[0]
Cases3 = range(n_daily_cases3)
for b in Banks:
    candidate_cases.iloc[candidate_cases.iloc[:, 39+b] > 0, 39+b] = 1
candidate_cases = candidate_cases.reset_index(drop = True)
candidate_cases


,transaction_id,description,Amount,category,date,month,customer_id,type,In_or_Out,bank_to,...,Utilities,day,time_spent,bank_A,bank_B,bank_C,bank_D,bank_E,Intrnl,to_investigate
0,327229,Center Parcs,498.97,Holiday,2024-08-02,August,10131,spending,paid_out,bank_D,...,0,306,1.00,1.0,0.0,0.0,1.0,0.0,0.0,1
1,327249,Restaurant Dinner - Date Night,92.81,Dining Out,2024-08-02,August,10220,spending,paid_out,bank_E,...,0,306,0.50,0.0,0.0,0.0,0.0,1.0,0.0,1
2,327302,Zara - Clothing Purchase,154.84,Shopping,2024-08-02,August,10518,spending,paid_out,bank_A,...,0,306,0.50,1.0,0.0,1.0,0.0,0.0,0.0,1
3,327320,Art Gallery and Museum,13.28,Entertainment,2024-08-02,August,10570,spending,paid_out,bank_E,...,0,306,0.25,1.0,0.0,0.0,0.0,1.0,0.0,1
4,327324,Chanel - Designer Fashion,2800.00,Shopping,2024-08-02,August,10570,spending,paid_out,bank_C,...,0,306,1.00,1.0,0.0,1.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,328599,Zara - Kids' Clothing,34.71,Shopping,2024-08-02,August,19122,spending,paid_out,bank_D,...,0,306,0.25,0.0,0.0,1.0,1.0,0.0,0.0,1
81,328603,Forever 21 - Accessories,16.23,Shopping,2024-08-02,August,19164,spending,paid_out,bank_D,...,0,306,0.25,0.0,0.0,0.0,1.0,1.0,0.0,1
82,328606,Charlotte Russe - Affordable Apparel,30.51,Shopping,2024-08-02,August,19264,spending,paid_out,bank_A,...,0,306,0.25,1.0,0.0,0.0,1.0,0.0,0.0,1
83,328639,Gap - Kids' Fashion,28.00,Shopping,2024-08-02,August,19689,spending,paid_out,bank_C,...,0,306,0.25,0.0,0.0,1.0,0.0,0.0,0.0,1


In [69]:
prob_2nd = xp.problem('prob_2nd')
prob_2nd.setControl('outputlog', 0) # suppress output

y = np.array([xp.var(vartype = xp.binary, name = 'y_{0}'.format(i+1))
            for i in Cases3], dtype = xp.npvar) # 0 or 1

int2 = np.array([xp.var(vartype = xp.integer, name = 'internal_{0}_{1}'.format(i+1, j+1))
            for i in Cases3 for j in Banks], dtype = xp.npvar).reshape(n_daily_cases3, n_banks) 
ext = np.array([xp.var(vartype = xp.binary, name = 'internal_{0}'.format(i+1))
            for i in Cases3], dtype = xp.npvar).reshape(n_daily_cases3)
prob_2nd.addVariable(y,int2,ext)

prob_2nd.addConstraint(int2[i,b] <= 2 * candidate_cases.iloc[i, 39+b] for i in Cases3 for b in Banks)
prob_2nd.addConstraint(ext[i] + xp.Sum(int2[i,b] for b in Banks)/2 == y[i] for i in Cases3)
prob_2nd.addConstraint(xp.Sum(int2[i,b]/2 * candidate_cases["time_spent"][i] for i in Cases3) <= teamsize[b] for b in Banks)
prob_2nd.setObjective(xp.Sum(ext[i] * ext_cost[candidate_cases["priority"][i]-1] + 
                             (1-y[i])*candidate_cases["Amount"][i] for i in Cases3), sense = xp.minimize)
prob_2nd.solve()
print(prob_2nd.getSolution(y))
decided_investigations = prob_2nd.getSolution(y)
print(len(decided_investigations))
candidate_cases["decided_investigations"] = decided_investigations
candidate_cases

prob_2nd.getSolution(ext)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
85


array([-0.,  0., -0., -0., -0., -0., -0., -0.,  0., -0., -0., -0., -0.,
       -0., -0.,  0., -0., -0.,  0.,  0., -0., -0., -0., -0., -0.,  0.,
       -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0., -0.,  0., -0., -0.,
        0., -0.,  0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,  0.,
       -0., -0., -0., -0., -0., -0., -0., -0., -0.,  0., -0., -0., -0.,
       -0.,  0., -0., -0.,  0.,  0., -0.,  0., -0., -0., -0.,  0.,  0.,
       -0.,  0., -0., -0., -0.,  0., -0.])

In [77]:
a = pd.DataFrame([[1,2,3],[3,4,5]])
print(a)
a[0] * a[1]

   0  1  2
0  1  2  3
1  3  4  5


0     2
1    12
dtype: int64

In [85]:
dumb = np.array([1,2,3])
testlol = 2
np.savetxt(f"testing {testlol}.csv", [p for p in zip(dumb)])